Intensity and $K_D$ Analysis
===

In [ ]:
####################################################################################
#                                                                                  #
# Leave these values blank to let the script determine them automatically.         #
# Only set them if something goes wrong!                                           #
#                                                                                  #
####################################################################################
flow_cell_id = ''
target_name = ''
neg_control_target_name = ''
all_channels = []
data_channel = ''
target_sequence_file = "/shared/targets.yml"
#nonneg_lda_weights_fpath = '/shared/yeast_beast_LDA_weights.txt'  # for microscope 3
nonneg_lda_weights_fpath = '/shared/bLDA_coef_nonneg.txt'  # for microscope 2 and 4

In [ ]:
from collections import defaultdict
import glob
import h5py
import itertools
import lomp
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pkg_resources
import progressbar
import random
import re
from scipy import stats
import sys
import yaml
from champ import misc, intensity, initialize, seqtools, gbtools, interactive
from champ.kd import fit_all_kds, saturated_at_concentration

try:
    matplotlib.style.use('flab')
except:
    pass

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
flow_cell_id = interactive.determine_flow_cell_id(flow_cell_id)
target_name = interactive.load_config_value('perfect_target_name', target_name)
neg_control_target_name = interactive.load_config_value('neg_control_target_name', neg_control_target_name)
all_channels = list(map(str, initialize.determine_channel_names('.'))) if not all_channels else all_channels
alignment_channel = interactive.load_config_value('alignment_channel', data_channel)
data_channel = interactive.determine_data_channel(all_channels, alignment_channel)

In [ ]:
read_name_dir = os.path.join('/shared', flow_cell_id, 'read_names')
bamfile_path = os.path.join('/shared', flow_cell_id, 'all_fastqs', 'genomic.bam')
read_name_kd_filename = os.path.join('results', 'cluster-data.h5')
read_names_by_seq_fpath = os.path.join(read_name_dir, 'read_names_by_seq.txt')

In [ ]:
with open(target_sequence_file) as f:
    targets = yaml.load(f)

target = targets[target_name]
neg_control_target = targets[neg_control_target_name]

print('Flow Cell ID: {}'.format(flow_cell_id))
print('Target {}: {}'.format(target_name, target))
print('Neg control target {}: {}'.format(neg_control_target_name, neg_control_target))
print('Channels: {}'.format(all_channels))
print('Protein channel: {}'.format(data_channel))
print('Output file: {}'.format(read_name_kd_filename))
print('Git commit used for this analysis: {}'.format(commit))

### Find all synthetic library sequences for this target

In [ ]:
interesting_seqs = set()
    
stretch = set()
for i in range(1, len(target)+1):
    stretch.update(seqtools.get_stretch_of_complement_seqs(target, i))
insertions = set()
for i in range(1, 3):
    insertions.update(seqtools.get_contiguous_insertion_seqs(target, i))
for i in range(1, 3):
    insertions.update(seqtools.get_insertion_seqs(target, i))   
deletions = set()
for i in range(1, 3):
    deletions.update(seqtools.get_deletion_seqs(target, i))
mismatches = set()
for i in range(1, 3):
    mismatches.update(seqtools.get_mismatch_seqs(target, i))
six_n_pam = seqtools.get_randomized_pam_seqs(target, 4, 6)
other_targets = set()
for s in targets.values():
    other_targets.add(s)

interesting_seqs.update(other_targets)
interesting_seqs.update(stretch)
interesting_seqs.update(insertions)
interesting_seqs.update(deletions)
interesting_seqs.update(mismatches)
interesting_seqs.update(six_n_pam)

print("Interesting sequences: %d" % len(interesting_seqs))

### Create Interesting Sequence Read Names File

In [ ]:
from champ.seqtools import build_interesting_sequences
interesting_read_names_filename = os.path.join(read_name_dir, 'interesting_{target_name}_reads_by_seq.txt'.format(target_name=target_name))
if os.path.exists(interesting_read_names_filename):
    # No need to recalculate, we can just load this from disk
    interesting_read_names = {}
    with open(interesting_read_names_filename) as f:
        for line in f:
            line = line.split("\t")
            sequence = line[0]
            read_names = line[1:]
            interesting_read_names[sequence] = read_names
else:
    interesting_read_names = build_interesting_sequences(read_names_by_seq_fpath, interesting_seqs)
    with open(interesting_read_names_filename, 'w') as f:
        for sequence, read_names in interesting_read_names.items():
            f.write("%s\t%s\n" % (sequence, "\t".join(read_names)))
            
print("Found read names for %d sequences of interest." % len(interesting_read_names))

Load Data
===

In [ ]:
all_read_name_fpath = os.path.join(read_name_dir, 'all_read_names.txt')
target_read_name_fpath = os.path.join(read_name_dir, 'target_{}_read_names.txt'.format(target_name.lower()))
perfect_target_read_name_fpath = os.path.join(read_name_dir, 'perfect_target_{}_read_names.txt'.format(target_name.lower()))
neg_control_target_read_name_fpath = os.path.join(read_name_dir, 'perfect_target_{}_read_names.txt'.format(neg_control_target_name.lower()))
phiX_read_name_fpath = os.path.join(read_name_dir, 'phix_read_names.txt')

all_read_names = set(line.strip() for line in open(all_read_name_fpath))
print("All read names: %d" % len(all_read_names))
target_read_names = set(line.strip() for line in open(target_read_name_fpath))
print("Target read names: %d" % len(target_read_names))
perfect_target_read_names = set(line.strip() for line in open(perfect_target_read_name_fpath))
print("Perfect target read names: %d" % len(perfect_target_read_names))
neg_control_target_read_names = set(line.strip() for line in open(neg_control_target_read_name_fpath))
print("Negative control read names: %d" % len(neg_control_target_read_names))
phiX_read_names = set(line.strip() for line in open(phiX_read_name_fpath))
print("Phix read names: %d" % len(phiX_read_names))

In [ ]:
h5_fpaths = glob.glob('*.h5')
h5_fpaths.sort(key=misc.parse_concentration)
for fpath in h5_fpaths:
    print misc.parse_concentration(fpath), fpath

In [ ]:
results_dirs = [
    os.path.join('results', 
                 os.path.splitext(os.path.basename(h5_fpath))[0])
    for h5_fpath in h5_fpaths
]
for d in results_dirs:
    print(d)

# Determine intensities of all aligned reads

In [ ]:
print 'Loading data...'
int_scores = intensity.IntensityScores(h5_fpaths)
int_scores.get_LDA_scores(results_dirs, nonneg_lda_weights_fpath)

In [ ]:
import time
print 'Normalizing data...'
int_scores.normalize_scores()
print 'Done normalizing.'

In [ ]:
int_scores.plot_aligned_images('br', 'o*')

In [ ]:
int_scores.plot_normalization_constants()

In [ ]:
int_scores.print_reads_per_channel()

In [ ]:
# The number of observations we require to consider a cluster valid enough for fitting.
# Clusters at the edge of a field of view might not be visible in every concentration due to
# random imperfections in the motion of the stage, and some fields of view might simply not
# align under each concentration.
minimum_observations_per_cluster = len(h5_fpaths) - 3

int_scores.build_good_read_names(5)
good_read_names = int_scores.good_read_names
good_perfect_read_names = perfect_target_read_names & good_read_names
print 'Good Reads:', len(good_read_names)
print 'Good Perfect Reads:', len(good_perfect_read_names)

In [ ]:
int_scores.build_score_given_read_name_given_channel()

In [ ]:
aligned_read_names = set()
for h5_fpath in h5_fpaths:
    for d in int_scores.scores[h5_fpath][data_channel].values():
        for read_name in d.keys():
            aligned_read_names.add(read_name)
print('Aligned reads in data channel: {}'.format(len(aligned_read_names))

### Save read names to disk

In [ ]:
string_dt = h5py.special_dtype(vlen=str)
aligned_read_names = list(sorted(aligned_read_names))

with h5py.File(read_name_kd_filename, 'w') as h5:
    h5.create_dataset('read_names', data=aligned_read_names, dtype=string_dt)

### Save sequence identifier intensities to disk

In [ ]:
read_name_intensities = defaultdict(list)
for h5_fpath in h5_fpaths:
    score_given_read_name = int_scores.score_given_read_name_in_channel[h5_fpath][data_channel]
    for read_name in aligned_read_names:
        intensity = score_given_read_name.get(read_name, np.nan)
        read_name_intensities[read_name].append(intensity)

intensity_matrix = []
for read_name in aligned_read_names:
    intensity_matrix.append(read_name_intensities[read_name])
    
intensity_matrix = np.array(intensity_matrix)

with h5py.File(read_name_kd_filename, 'a') as h5:
    h5.create_dataset('intensities', data=intensity_matrix)

# Determine what a saturated cluster looks like

In [ ]:
# Find the KD of the perfect target so we can determine at what concentrations the clusters
# should be saturated
sequence_read_name_intensities = defaultdict(list)
for sequence, read_names in interesting_read_names.items():
    for read_name in aligned_read_names:
        if read_name not in read_name_intensities:
            continue
        sequence_read_name_intensities[sequence].append(read_name_intensities[read_name])

all_concentrations = [misc.parse_concentration(h5_fpath) for h5_fpath in h5_fpaths]
sequence, kd, kd_uncertainty, yint, perfect_delta_y, counts = list(fit_all_kds({target: sequence_read_name_intensities[target]}, all_concentrations, process_count=1))[0]
print("Perfect target KD is %.1f +/- %.3f nM" % (kd, kd_uncertainty))


# Determine the median intensity of a saturated cluster
saturated = saturated_at_concentration(kd)
saturated_indexes = [index for index, concentration in enumerate(all_concentrations) if concentration > saturated]
if not saturated_indexes:
    # this should never happen, but we'll try to take something reasonable
    print("Warning: perfect target sequence probably did not saturate its target!")
    saturated_indexes = len(all_concentrations) - 1
    
saturated_intensities = []
for intensity_gradient in sequence_read_name_intensities[target]:
    for index in saturated_indexes:
        try:
            value = intensity_gradient[index]
            if not np.isnan(value):
                saturated_intensities.append(value)
        except IndexError:
            continue
median_saturated_intensity = np.median(saturated_intensities)
print("Median saturated intensity: %d (N=%d)" % (median_saturated_intensity, len(saturated_intensities)))

# Determine the KDs of all synthetic sequences

In [ ]:
sequence_kds = {}
with progressbar.ProgressBar(max_value=len(sequence_read_name_intensities)) as pbar:
    for sequence, kd, kd_uncertainty, yint, delta_y, count in pbar(fit_all_kds(sequence_read_name_intensities, all_concentrations, process_count=8, delta_y=median_saturated_intensity)):
        if count >= 5:
            sequence_kds[sequence] = kd, kd_uncertainty, yint, delta_y, count
print("Found KDs for %d of %d sequences of interest" % (len(sequence_kds), len(sequence_read_name_intensities)))

# Save the KDs to disk

In [ ]:
string_dt = h5py.special_dtype(vlen=str)
kd_dt = np.dtype([('sequence', string_dt),
                  ('kd', np.float),
                  ('kd_uncertainty', np.float),
                  ('y_intercept', np.float),
                  ('delta_y', np.float),
                  ('count', np.int32)])
with h5py.File(read_name_kd_filename, 'a') as h5:
    dataset = h5.create_dataset('synthetic-kds', (len(sequence_kds),), dtype=kd_dt)
    dataset[...] = [(sequence, kd, kd_uncertainty, yint, delta_y, count) for sequence, (kd, kd_uncertainty, yint, delta_y, count) in sequence_kds.items()]

# Statistical Overview

In [ ]:
histogram_kds = [k[0] for k in sequence_kds.values()]
histogram_counts = [k[4] for k in sequence_kds.values()]
fig, (kd_ax, count_ax) = plt.subplots(1, 2, figsize=(16,6))
kd_ax.hist(histogram_kds, bins=100);
kd_ax.set_ylabel("Count")
kd_ax.set_xlabel("$K_D$ (nM)")
kd_ax.set_title("Histogram of Synthetic Target KDs")

count_median = np.median(histogram_counts)
count_iqr = stats.iqr(histogram_counts)
count_ax.hist(histogram_counts, bins=20, range=(0, int(count_median+count_iqr*5)))
count_ax.set_ylabel("Count")
count_ax.set_xlabel("Clusters per Sequence")
count_ax.set_title("Representation of Synthetic Sequences")
fig.tight_layout()